In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("earthquakes database NEIC.csv")

In [ ]:
# Display the first 10 rows of the DataFrame
df.head(10)

In [ ]:
df.info()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
display("Missing Values:")
display(missing_values)


In [ ]:
# Check for duplicate entries
duplicate_entries = df.duplicated().sum()
display("Number of Duplicate Entries:", duplicate_entries)

In [ ]:
# Convert data types if needed
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

In [ ]:
# Drop rows with missing latitude or longitude values
df = df.dropna(subset=['Latitude', 'Longitude'])

In [ ]:
# Display the first few rows of the cleaned DataFrame
display("Cleaned DataFrame:")
display(df.head())

In [ ]:
# Calculate basic summary statistics for numerical columns: elevation, latitude, and longitude
numerical_columns_summary = df[['Magnitude', 'Latitude', 'Longitude']].describe()

In [ ]:
# Calculate range for each numerical column as the difference between max and min
numerical_columns_range = df[['Magnitude', 'Latitude', 'Longitude']].max() - df[['Magnitude', 'Latitude', 'Longitude']].min()

In [ ]:
# Append range to the summary statistics
numerical_columns_summary.loc['range'] = numerical_columns_range

In [ ]:
numerical_columns_summary

In [ ]:
# Calculate basic summary statistics for numerical columns: elevation, latitude, and longitude
numerical_columns_summary = df[['Magnitude', 'Latitude', 'Longitude']].describe()

# Calculate range for each numerical column as the difference between max and min
numerical_columns_range = df[['Magnitude', 'Latitude', 'Longitude']].max() - df[['Magnitude', 'Latitude', 'Longitude']].min()

# Append range to the summary statistics
numerical_columns_summary.loc['range'] = numerical_columns_range

numerical_columns_summary

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Explore the distribution of categorical variables: Magnitude_Type
magnitude_type_distribution = df['Magnitude Type'].value_counts()

(magnitude_type_distribution)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Visualization: Magnitude vs Depth
fig, ax = plt.subplots(figsize=(7, 6))

# Scatter plot of magnitude vs depth
sns.scatterplot(x='Magnitude', y='Depth', data=df, ax=ax)
ax.set_title('Magnitude vs Depth of Earthquakes')
ax.set_xlabel('Magnitude (m)')
ax.set_ylabel('Depth')

plt.tight_layout()
plt.show()

In [ ]:
pip install pandas geopy

In [ ]:
pip install tqdm

In [ ]:
pip install bokeh


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar
from bokeh.transform import transform
import numpy as np
import pandas as pd

# Ensure output is displayed in the notebook
output_notebook()

# Load your data (replace 'your_data.csv' with your actual data file)
# Assuming your data has columns: 'Latitude', 'Longitude', and 'Magnitude'
# Replace this line with your actual data loading code
df = pd.read_csv('earthquakes database NEIC.csv')

# Convert lat/lon to Web Mercator coordinates
def wgs84_to_web_mercator(df, lon="Longitude", lat="Latitude"):
    k = 6378137
    df["Longitude"] = df[lon] * (k * np.pi / 180.0)
    df["Latitude"] = np.log(np.tan((90 + df[lat]) * np.pi / 360.0)) * k
    return df

df = wgs84_to_web_mercator(df)

# Define the color mapper: linear mapping from elevations to a color palette
color_mapper = LinearColorMapper(palette="Viridis256", low=df['Magnitude'].min(), high=df['Magnitude'].max())

# Set up the figure
p = figure(title="Volcano Locations by Elevation", x_axis_type="mercator", y_axis_type="mercator",
           x_axis_label="Longitude", y_axis_label="Latitude", sizing_mode="scale_width", height=400)

# Add points using 'circle' glyphs, color by elevation
p.circle(x="Longitude", y="Latitude", source=ColumnDataSource(df),
         size=5, color=transform('Magnitude', color_mapper), alpha=0.6)

# Show the plot
show(p)


In [ ]:
pip install folium


In [ ]:
import folium
import pandas as pd
import numpy as np  # Don't forget to import NumPy!

# Load your data (replace 'your_data.csv' with your actual data file)
# Assuming your data has columns: 'Latitude', 'Longitude', and 'Magnitude'
# Replace this line with your actual data loading code
df = pd.read_csv('earthquakes database NEIC.csv')

# Convert lat/lon to Web Mercator coordinates
def wgs84_to_web_mercator(row):
    lon, lat = row['Longitude'], row['Latitude']
    k = 6378137
    x = lon * (k * np.pi / 180.0)
    y = np.log(np.tan((90 + lat) * np.pi / 360.0)) * k
    return x, y

df['x'], df['y'] = zip(*df.apply(wgs84_to_web_mercator, axis=1))

# Create a Folium map centered at a specific location (you can adjust this)
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Add circle markers to the map
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"Magnitude: {row['Magnitude']}"
    ).add_to(m)

# Display the map
m


In [ ]:
import folium
import pandas as pd
import numpy as np

# Load your data (replace 'your_data.csv' with your actual data file)
# Assuming your data has columns: 'Latitude', 'Longitude', and 'Magnitude'
# Replace this line with your actual data loading code
df = pd.read_csv('earthquakes database NEIC.csv')

# Convert lat/lon to Web Mercator coordinates
def wgs84_to_web_mercator(row):
    lon, lat = row['Longitude'], row['Latitude']
    k = 6378137
    x = lon * (k * np.pi / 180.0)
    y = np.log(np.tan((90 + lat) * np.pi / 360.0)) * k
    return x, y

df['x'], df['y'] = zip(*df.apply(wgs84_to_web_mercator, axis=1))

# Define color gradient (yellow to red) based on Magnitude
color_scale = folium.LinearColormap(colors=['yellow', 'red'], vmin=df['Magnitude'].min(), vmax=df['Magnitude'].max())

# Create a Folium map centered at a specific location (you can adjust this)
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Separate data points by magnitude (higher magnitude on top)
high_magnitude_df = df[df['Magnitude'] >= df['Magnitude'].median()]
low_magnitude_df = df[df['Magnitude'] < df['Magnitude'].median()]

# Adjust the size scaling factor for better visual balance
size_scaling_factor = 1.5  # You can tweak this value

# Add high magnitude data points first (they will be visually on top)
for _, row in high_magnitude_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude'] * size_scaling_factor,
        color=color_scale(row['Magnitude']),
        fill=True,
        fill_color=color_scale(row['Magnitude']),
        fill_opacity=0.6,
        popup=f"Magnitude: {row['Magnitude']:.2f}"
    ).add_to(m)

# Add low magnitude data points afterward
for _, row in low_magnitude_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude'] * size_scaling_factor,
        color=color_scale(row['Magnitude']),
        fill=True,
        fill_color=color_scale(row['Magnitude']),
        fill_opacity=0.6,
        popup=f"Magnitude: {row['Magnitude']:.2f}"
    ).add_to(m)

# Add color legend
color_scale.caption = 'Magnitude'
color_scale.add_to(m)

# Display the map
m


In [ ]:
import folium
import pandas as pd
import numpy as np

# Load your data (replace 'your_data.csv' with your actual data file)
# Assuming your data has columns: 'Latitude', 'Longitude', and 'Magnitude'
# Replace this line with your actual data loading code
df = pd.read_csv('earthquakes database NEIC.csv')

# Convert lat/lon to Web Mercator coordinates
def wgs84_to_web_mercator(row):
    lon, lat = row['Longitude'], row['Latitude']
    k = 6378137
    x = lon * (k * np.pi / 180.0)
    y = np.log(np.tan((90 + lat) * np.pi / 360.0)) * k
    return x, y

df['x'], df['y'] = zip(*df.apply(wgs84_to_web_mercator, axis=1))

# Define color gradient (yellow to red) based on Magnitude
color_scale = folium.LinearColormap(colors=['yellow', 'red'], vmin=df['Magnitude'].min(), vmax=df['Magnitude'].max())

# Create a Folium map centered at a specific location (you can adjust this)
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Separate data points by magnitude (higher magnitude on top)
high_magnitude_df = df[df['Magnitude'] >= df['Magnitude'].median()]
low_magnitude_df = df[df['Magnitude'] < df['Magnitude'].median()]

# Adjust the size scaling factor for better visual balance
size_scaling_factor = 1.5  # You can tweak this value

# Add low magnitude data points first
for _, row in low_magnitude_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude'] * size_scaling_factor,
        color=color_scale(row['Magnitude']),
        fill=True,
        fill_color=color_scale(row['Magnitude']),
        fill_opacity=0.6,
        popup=f"Magnitude: {row['Magnitude']:.2f}"
    ).add_to(m)

# Add high magnitude data points afterward
for _, row in high_magnitude_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude'] * size_scaling_factor,
        color=color_scale(row['Magnitude']),
        fill=True,
        fill_color=color_scale(row['Magnitude']),
        fill_opacity=0.6,
        popup=f"Magnitude: {row['Magnitude']:.2f}"
    ).add_to(m)

# Add color legend
color_scale.caption = 'Magnitude'
color_scale.add_to(m)

# Display the map
m
